In [3]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import sys
from functools import partial
import glob
from tqdm import tqdm
from itertools import chain
from collections import defaultdict
import datetime
import numpy as np
np.set_printoptions(precision=16,threshold=np.inf)
import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

sys.path.append('/home/shenqi/Master_thesis/Shenqi-MasterThseis/Lib')

# import metavision_ml_copy.data
from metavision_ml_copy.data import SequentialDataLoader
from metavision_ml_copy.data import box_processing as box_api
from metavision_ml_copy.detection.losses import DetectionLoss
from metavision_ml_copy.detection.feature_extractors import EGRUReLu_resnet_sparse, EMinRNN_ReLu, ELSTMReLu_resnet_sparse
from metavision_ml_copy.detection.single_stage_detector import SingleStageDetector
from metavision_ml_copy.metrics.coco_eval import CocoEvaluator

sys.path.append('/home/shenqi/Master_thesis/Shenqi-MasterThseis/code')
# from metavision_ml.metrics.coco_eval import evaluate_detection
from utils.PropheseeToolbox.src.metrics.coco_eval import evaluate_detection
from utils.PropheseeToolbox.src.io.box_loading import reformat_boxes
from utils.PropheseeToolbox.src.io.box_filtering import filter_boxes
from metavision_sdk_core import EventBbox

torch.manual_seed(0)
np.random.seed(0)
scaler = torch.cuda.amp.GradScaler()


dataset_type = 'gen4'

if dataset_type == 'gen1':
    dataset_path = '/media/shenqi/data/Gen1_multi_timesurface_FromDat'
    # dataset_path = '/media/shenqi/data_0/Gen1_multi_timesurface_small'
elif dataset_type == 'gen4':
    dataset_path = '/media/shenqi/data/Gen4_multi_timesurface_FromDat'


label_map_path = os.path.join(dataset_path, 'label_map_dictionary.json')
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')
test_path = os.path.join(dataset_path, 'test')

train_gt_file_paths = sorted(glob.glob(train_path+'/*.npy'))
train_data_file_paths = sorted(glob.glob(train_path+'/*.h5'))
test_gt_file_paths = sorted(glob.glob(test_path+'/*.npy'))
test_data_file_paths = sorted(glob.glob(test_path+'/*.h5'))
val_gt_file_paths = sorted(glob.glob(val_path+'/*.npy'))
val_data_file_paths = sorted(glob.glob(val_path+'/*.h5'))

print('train:',len(train_gt_file_paths), len(train_data_file_paths))
print('val:',len(val_gt_file_paths), len(val_data_file_paths))
print('test:',len(test_gt_file_paths), len(test_data_file_paths))

if dataset_type == 'gen1':
    wanted_keys = [ 'car', 'pedestrian']
    height, width = 240, 304
    min_box_diag_network = 30
elif dataset_type == 'gen4':
    wanted_keys = ['pedestrian', 'two wheeler', 'car']
    height, width = 360, 640
    min_box_diag_network = 60

class_lookup = box_api.create_class_lookup(label_map_path, wanted_keys)
print(class_lookup)
label_map = ['background'] + wanted_keys
files = glob.glob(os.path.join(train_path, "*.h5"))
files_val = glob.glob(os.path.join(val_path, "*.h5"))
files_test = glob.glob(os.path.join(test_path, "*.h5"))

preprocess_function_name = "multi_channel_timesurface" 
delta_t = 50000
channels = 6  # histograms have two channels
num_tbins = 1
batch_size = 24
max_incr_per_pixel = 5
array_dim = [num_tbins, channels, height, width]


load_boxes_fn = partial(box_api.load_boxes, num_tbins=num_tbins, class_lookup=class_lookup, min_box_diag_network=min_box_diag_network)

seq_dataloader = SequentialDataLoader(files, delta_t, preprocess_function_name, array_dim, load_labels=load_boxes_fn,
                                      batch_size=batch_size, num_workers=8, padding=True, preprocess_kwargs={"max_incr_per_pixel": max_incr_per_pixel})

seq_dataloader_val = SequentialDataLoader(files_val, delta_t, preprocess_function_name, array_dim, load_labels=load_boxes_fn,
                                      batch_size=batch_size, num_workers=8, padding=True,preprocess_kwargs={"max_incr_per_pixel": max_incr_per_pixel})
seq_dataloader_test = SequentialDataLoader(files_test, delta_t, preprocess_function_name, array_dim, load_labels=load_boxes_fn,
                                      batch_size=batch_size, num_workers=8, padding=True,preprocess_kwargs={"max_incr_per_pixel": max_incr_per_pixel})
print(len(seq_dataloader))
print(len(seq_dataloader_val))
print(len(seq_dataloader_test))



train: 1360 680
val: 128 128
test: 240 120
[ 1  2  3 -1 -1 -1 -1]
34800
7200
6000


In [4]:
threshold_up = 1
threshold_down = 0.5

output_dt_delete_list = [[] for i in range(batch_size)]
output_gt_delete_list = [[] for i in range(batch_size)]
with tqdm(total=len(seq_dataloader_test), desc=f'Validation',ncols=120) as pbar:
    cnt = [0] * batch_size      
    for data in seq_dataloader_test:
        pbar.update(1)
        batch = data["inputs"]
        for index, video_info in enumerate(data['video_infos']):
            cnt[index] += 1
            file_root_path = str(video_info[0]).split(' ')[2].split('\n')[0].split('.')[0].split('/')[-1]
            dt_file_path = './dt_results_convnn/Gen4_raw/' + file_root_path + '_dt_bbox.npy'
            gt_file_path = './dt_results_convnn/Gen4_raw/' + file_root_path + '_GroundTruth_bbox.npy'
            dt_file = np.load(dt_file_path)
            gt_file = np.load(gt_file_path)
            
            dt_box_interval = dt_file[ (dt_file['t'] >= (cnt[index]-1) * 50000) & (dt_file['t'] < cnt[index]*50000)]
            gt_box_interval = gt_file[ (gt_file['t'] >= (cnt[index]-1) * 50000) & (gt_file['t'] < cnt[index]*50000)]
            
            origin_inputs = batch[0][index]
            
            for dt_box_events in dt_box_interval:
                dt_topleft_x = np.clip(dt_box_events["x"], 0, width - 1).astype('int')
                dt_topleft_y = np.clip(dt_box_events["y"], 0, height - 1).astype('int')
                dt_botright_x = np.clip(dt_box_events["x"] + dt_box_events["w"], 0, width - 1).astype('int')
                dt_botright_y = np.clip(dt_box_events["y"] + dt_box_events["h"], 0, height - 1).astype('int')
                
                dt_event_ave = torch.where(origin_inputs[:,dt_topleft_y:dt_botright_y, dt_topleft_x:dt_botright_x] == 0, torch.zeros_like(origin_inputs[:,dt_topleft_y:dt_botright_y, dt_topleft_x:dt_botright_x]),
                                        torch.ones_like(origin_inputs[:,dt_topleft_y:dt_botright_y, dt_topleft_x:dt_botright_x])).mean()
                if dt_event_ave < threshold_up and dt_event_ave > threshold_down:
                    output_dt_delete_list[index].append(dt_box_events)
            
            for gt_box_events in gt_box_interval:
                gt_topleft_x = np.clip(gt_box_events["x"], 0, width - 1).astype('int')
                gt_topleft_y = np.clip(gt_box_events["y"], 0, height - 1).astype('int')
                gt_botright_x = np.clip(gt_box_events["x"] + gt_box_events["w"], 0, width - 1).astype('int')
                gt_botright_y = np.clip(gt_box_events["y"] + gt_box_events["h"], 0, height - 1).astype('int')
                
                gt_event_ave = torch.where(origin_inputs[:,gt_topleft_y:gt_botright_y, gt_topleft_x:gt_botright_x] == 0, torch.zeros_like(origin_inputs[:,gt_topleft_y:gt_botright_y, gt_topleft_x:gt_botright_x]),
                                        torch.ones_like(origin_inputs[:,gt_topleft_y:gt_botright_y, gt_topleft_x:gt_botright_x])).mean()
                if gt_event_ave < threshold_up and gt_event_ave > threshold_down:
                    output_gt_delete_list[index].append(gt_box_events)
            


            if cnt[index] == 1200:
                dt_file_name_path = './dt_results_convnn/Gen4_1/' + file_root_path + '_dt_bbox.npy'
                np.save(dt_file_name_path, np.array(output_dt_delete_list[index], dtype = EventBbox)) 
                cnt[index] = 0
                output_dt_delete_list[index] = []
                
                gt_file_name_path = './dt_results_convnn/Gen4_1/' + file_root_path + '_GroundTruth_bbox.npy'
                np.save(gt_file_name_path, np.array(output_gt_delete_list[index], dtype = EventBbox)) 
                cnt[index] = 0
                output_gt_delete_list[index] = []
            


Validation: 100%|███████████████████████████████████████████████████████████████████| 6000/6000 [32:18<00:00,  3.09it/s]


In [5]:

dt_detections = defaultdict(list)
gt_detections = defaultdict(list)


detction_output_path ='./dt_results_convnn/Gen4_1/'
    


test_gt_files = sorted(glob.glob(detction_output_path + '/*_GroundTruth_bbox.npy'))   
gt_boxes_list = [np.load(p) for p in test_gt_files]
test_dt_files = sorted(glob.glob(detction_output_path + '/*_dt_bbox.npy'))  
result_boxes_list = [np.load(p) for p in test_dt_files]

gt_boxes_list = [reformat_boxes(p) for p in gt_boxes_list]
result_boxes_list = [reformat_boxes(p) for p in result_boxes_list]


min_box_diag,min_box_side = 60, 20
filter_boxes_fn = lambda x:filter_boxes(x, 500000, min_box_diag, min_box_side)
gt_boxes_list = map(filter_boxes_fn, gt_boxes_list)
result_boxes_list = map(filter_boxes_fn, result_boxes_list)

evaluate_detection(gt_boxes_list, result_boxes_list,classes=label_map, height=height, width=width,time_tol=40000)



creating index...
index created!
Loading and preparing results...
DONE (t=0.25s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.87s).
Accumulating evaluation results...
DONE (t=0.95s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.173
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.367
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.321
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.252
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.386
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.

In [6]:
'''
0.01: Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.037
        Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.095
        Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.023
        Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
        Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.023
        Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.043
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.071
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.128
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.170
        Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.151
        Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.168
        
        
        
0.05:Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.244
        Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.458
        Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.222
        Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
        Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.206
        Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.247
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.263
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.372
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.379
        Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.343
        Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.393
        
0.1:Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.371
                Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.608
                Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.390
                Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
                Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.358
                Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.410
                Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.384
                Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.497
                Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.499
                Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
                Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.481
                Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.548
        
0.5Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.452
        Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.766
        Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.463
        Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
        Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.430
        Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.531
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.351
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.592
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.601
        Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
        Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.586
        Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.656
        
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.173
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.367
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.321
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.252
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.386
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.533
'''


'\n0.01: Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.037\n        Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.095\n        Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.023\n        Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000\n        Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.023\n        Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.043\n        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.071\n        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.128\n        Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.170\n        Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000\n        Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.151\n        Average Recall     (A